# Using CBFs for forbidden zones

To deal with the forbidden zones we can utilize a CBF with half plane constraints
and solve it with quadratic programming.

$$
\begin{aligned}
U_1 &= \{a_1^\top u + b_1 \leq 0\} \\
U_2 &= \{a_2^\top u + b_2 \leq 0\} \\
U_b &= U_{b,1} \cap U_{b,2}
\end{aligned}
$$

Which can than be formulated into

$$
\begin{aligned}
\min_{\substack{u \\\kappa_i \in U_{b,i}}} & \quad u^\top H u + p \delta \\
\text{s.t.} & \quad I u = \kappa \\
            & \quad u_\text{min} \leq u \leq u_\text{max}
\end{aligned}
$$

In [1]:
import numpy as np
import scipy as sp
import qpsolvers as qp
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, FloatSlider, FloatText

$$
\begin{aligned}
\min_x & \quad \frac{1}{2} x^\top P x + q^\top x \\
\text{s.t.} & \quad G x \leq h \\
            & \quad A x = b \\
            & \quad x_\text{min} \leq x \leq x_\text{max}
\end{aligned}
$$



In [2]:
def solve_qp(x1_d, x2_d):
    kargs = {}
    # Define the 2-Dimentional QP Problem with 2 slack variables
    kargs['P'] = np.array(
        [
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1e5, 0],
            [0, 0, 0, 1e5]
        ], dtype=np.float32)
    kargs['q'] = np.array([x1_d, x2_d, 0, 0], dtype=np.float32)

    kargs['A'] = np.array(
        [
            [1, 0, 1, 0],
            [0, 1, 0, 1],
        ], dtype=np.float32)
    kargs['b'] = np.array([x1_d, x2_d], dtype=np.float32)

    n = 3
    a1 = np.arctan2(x2_d, x1_d)
    alpha = np.linspace(a1 - np.pi / 2, a1 + np.pi / 2, n)
    G = np.concatenate((
            np.reshape(np.cos(alpha), (np.shape(alpha)[0], 1)),
            np.reshape(np.sin(alpha), (np.shape(alpha)[0], 1)),
            np.zeros((n, 2))
        ), axis=1)
    G = G[np.where(~np.any(np.isinf(G), axis=1))[0]]
    G = G[np.where(~np.any(np.abs(G) > 1e3, axis=1))[0]]
    kargs['G'] = np.array(G, dtype=np.float32)

    h = np.ones(np.shape(G)[0])
    kargs['h'] = np.array(h, dtype=np.float32)

    kargs['ub'] = np.array([np.inf, np.inf,  np.inf, np.inf], dtype=np.float32)
    kargs['lb'] = np.array([-np.inf, -np.inf, -np.inf, -np.inf], dtype=np.float32)
    problem = qp.Problem(**kargs)
    solution = qp.solve_problem(problem, solver='osqp', verbose=False)
    print(f"Solution {solution.x} \n norm(x1 + x2): {np.linalg.norm((solution.x[0], solution.x[1]))}")

    # Plot the solution
    a = np.linspace(0, 2*np.pi, 100)
    plt.plot(np.cos(a), np.sin(a), 'r')
    plt.scatter(x=[x1_d], y=[x2_d], c='b', s=100, marker='x')
    plt.plot(solution.x[0], solution.x[1], 'bo')
    plt.axis('equal')
    plt.grid(True)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.xlim(-2.0, 2.0)
    plt.ylim(-2.0, 2.0)
    plt.show()

In [3]:
x1_d_input = FloatSlider(value=-0.4, min=-3, max=3, description='x0_(1)):')
x2_d_input = FloatSlider(value=1.2, min=-3, max=3, description='x0_(2):')

# Create the interactive plot
interact(solve_qp, x1_d=x1_d_input, x2_d=x2_d_input)

interactive(children=(FloatSlider(value=-0.4, description='x0_(1)):', max=3.0, min=-3.0), FloatSlider(value=1.…

<function __main__.solve_qp(x1_d, x2_d)>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatText

# Define the function to update the plot
def update_plot(a1, x0_x, x0_y):
    # Create an array of values from 0 to 2*pi
    a = np.linspace(0, 2*np.pi, 100)

    # Plot a circle with radius 1
    plt.plot(np.cos(a), np.sin(a), 'k', linewidth=3)

    # Define the coefficients of the linear equation
    A = np.array([[np.cos(a1), np.sin(a1)]])
    b = np.array([1])
    x = np.linalg.lstsq(A, b, rcond=None)[0]

    # Create a range of x values
    x_values = np.linspace(-3, 3, 100)

    # Define the point
    x0 = np.array([x0_x, x0_y])

    # Check if the point is below or above the line
    if np.dot(A, x0) < 1:
        plt.scatter(x0[0], x0[1], color='green', label='Below line')
    else:
        plt.scatter(x0[0], x0[1], color='red', label='Above line')

    # Solve for y values
    y1 = (b[0] - A[0,0]*x_values) / A[0,1]

    # Plot the line
    plt.plot(x_values, y1, 'r', linewidth=2, label='Linear Equation')

    # Add labels and grid
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.axis('equal')
    plt.grid(True)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

# Create sliders for a1 and input boxes for x0 values
a1_slider = FloatSlider(value=4*np.pi/5, min=-2*np.pi, max=2*np.pi, step=0.01, description='a1:')
x0_x_input = FloatSlider(value=-0.4, min=-3, max=3, description='x0_(1)):')
x0_y_input = FloatSlider(value=1.2, min=-3, max=3, description='x0_(2):')

# Create the interactive plot
interact(update_plot, a1=a1_slider, x0_x=x0_x_input, x0_y=x0_y_input)

interactive(children=(FloatSlider(value=2.5132741228718345, description='a1:', max=6.283185307179586, min=-6.2…

<function __main__.update_plot(a1, x0_x, x0_y)>